In [71]:
import serial
import time
import pandas as pd
import scipy
from scipy import signal
import numpy as np
import math
from statistics import mean

In [72]:
import pickle
global clf
clf = pickle.load(open('testmodel.pkl','rb'))

In [74]:
def event_detection(data):
    '''
    Detects devices being turned On
    Returns the timestamp indices where devices are turned on
    '''
    data = scipy.signal.medfilt(data, kernel_size=None)
    dbydx = np.gradient(data)
    on_events = []
    for idx,x in enumerate(dbydx):
        if x > THRSHLD:
            on_events.append(idx)
    for x in on_events:
        for y in range(x+1,x+91): # to remove false turn on signatures, give delay for transition
            if y in on_events:
                on_events.remove(y)
    return on_events

In [257]:
def get_signal (length):
    '''
    Takes the length of signature required,
    returns signature and timestamps
    
    '''
    COUNT = 0
    buffer = np.array([])
    time_buffer = np.array([])
    while COUNT!= length :
        b = ser.readline()         # read a byte string
        string_n = b.decode(errors='ignore')  # decode byte string into Unicode  
        try:
            val, timestamp = string_n.split(sep = ',')
            val = float(val)
            timestamp = pd.to_datetime(timestamp)
            buffer = np.append(buffer, val) # helps getting data in seconds
            time_buffer = np.append(time_buffer,timestamp)
            COUNT += 1
        except:
            print('read failed, retrying!')
    print(string_n)
    return buffer,time_buffer

In [258]:
def predict_clf(data): #unused
    global clf
    guess = clf.predict(data)
    acc = clf.predict_proba(data).max()
    print(f'Device turned on at {sig_ts} and is probably {guess}, Acc.: {acc}') 

In [259]:
def log_event(guess,sig_ts,rise,sig):
    '''
    Stores the following:
    1. Signature of appliance,
    2. Time-stamp of turning ON
    3. Name
    4. Rise in consumption
    
    '''
    global logs_df
    global prev_mean #simple case
    logs_df = logs_df.append(pd.DataFrame({'Sig':[sig],
                                           'Name':guess,
                                           'Turned ON':sig_ts,
                                           'Init Rise':rise,
                                           'Curr Rise':rise,
                                           'Fall':prev_mean,
                                           'Turned OFF':0},index=[0]),ignore_index=True)

In [268]:
def track_device(num,val,ts):
    #simple case
    global logs_df
    curr_rise = logs_df['Curr Rise'][num]
    print(f'current rise {curr_rise}')
    init_rise = logs_df['Init Rise'][num]
    fall = logs_df['Fall'][num]
    print(f'fall {fall}')
    offset = curr_rise - val # gives current rise or fall 
    print(f'offset {offset}')
    curr_rise = curr_rise - offset # update
    print(f'newrise {curr_rise}')
    logs_df.loc[[num],['Curr Rise']] = curr_rise
    if math.isclose(curr_rise,fall,abs_tol=abs(init_rise*1/100)):
        print('yeaaaaaaaaaaa')
        logs_df.loc[[num],['Turned OFF']] = ts
        return 1
    else:
        print('nooooooooo')
        return 0

In [269]:
global logs_df
logs_df = pd.DataFrame([],columns = ['Sig','Name','Turned ON','Init Rise','Curr Rise','Fall','Turned OFF'])

In [270]:
SIG_SIZE = 90 # 1 sec contains 30 samples ; 3 seconds 90 samples.
THRSHLD = 30 # Rise in kWH to be detected for classification
BUFFER_SIZE = 360 # signal buffer size 
i = 1 #test purpose iterator
global prev_mean
prev_mean = 0 #maybe declare global
# These are for creating local database and saving signatures!!!
prev_guess=''
global active_appliances
active_appliances = 0

In [273]:
ser = serial.Serial('COM3', 115200) # remove later
time.sleep(2)


# Read and record the data
for i in range(10):
    buffer, time_buffer = get_signal(BUFFER_SIZE)
    events = event_detection(buffer) # if no on events, check for off events
    if len(events)>0:
        start = time.time()
        print(f'{len(events)} events detected')
        for ev in events:
            sig = buffer[ev:ev+SIG_SIZE]
            if len(sig) != SIG_SIZE:
                print('signal not adequate, shifting buffer')
                buffer = np.roll(buffer, -SIG_SIZE)
                time_buffer = np.roll(time_buffer, -SIG_SIZE)
                temp_buffer, temp_time_buffer = get_signal(SIG_SIZE)
                buffer[BUFFER_SIZE-SIG_SIZE:] = temp_buffer
                time_buffer[BUFFER_SIZE-SIG_SIZE:] = temp_time_buffer
                events = event_detection(buffer)
                print(f'{len(events)} events detected on reconstructing')
            if len(events)>0:
                for ev in events:
                    sig = buffer[ev:ev+SIG_SIZE]
                    if sig.mean() > prev_mean:
                        sig = sig - prev_mean
                        sig_ts = time_buffer[ev]
                        sig = sig.reshape((sig.shape[0]//SIG_SIZE,SIG_SIZE))
                        rise = buffer[ev+SIG_SIZE:].mean()
                        guess = clf.predict(sig)
                        end = time.time()
                        acc = clf.predict_proba(sig).max()
                        active_appliances += 1
                        print(f'{guess} turned on at {sig_ts} with rise {rise}, Acc.: {acc}')
                        if acc > 0.80:
                            if prev_guess == guess:
                                print(f'It seems the {guess} was detected again at {sig_ts}, if true confirm in {local_name}, but if different, name it\n')
                                name = input('Enter the name!\n')
                                if name == guess:
                                    # for decremening active_devices and clubbing
                                    # save signature
                                    pass
                                else:
                                    # save signature
                                    pass
                                prev_guess = guess
                                log_event(name,sig_ts,rise,sig)
                                #here might want to decrement active_devices and remove start time from it and club it by comparing prev and current guess!
                            else:
                                name = input(f'An appliance  was detected at {sig_ts}! Some people named it as {guess}, would you like to name it?\n')
                                prev_guess = guess
                                log_event(name,sig_ts,rise,sig)
                        else:
#                             name = input(f'An unrecognized device was detected at {sig_ts}, please name it\n')
                            name = guess
                            prev_guess = guess
                            log_event(name,sig_ts,rise,sig)
                print(f'Total time taken {end-start:.3f} s')
                break # breaks previous 'for-loops' of events 
        

    else:
        prev_mean = buffer.mean()
        for aplnc in range(active_appliances):
            # update values of rise to find off event
            curr_val = prev_mean
            m = track_device(aplnc,curr_val,time_buffer[-1])
        if m == 1:
            print('*****')
            break #breaks i iterator
        

290.00,2020-11-4 2:39:24.650

1 events detected
['washing machine'] turned on at 2020-11-04 02:39:17.150000 with rise 290.5, Acc.: 0.62
Total time taken 0.039 s
288.00,2020-11-4 2:39:34.604

current rise 290.5
fall 0
offset 1.2027777777777828
newrise 289.2972222222222
nooooooooo
291.00,2020-11-4 2:39:44.560

current rise 289.2972222222222
fall 0
offset -1.0361111111110972
newrise 290.3333333333333
nooooooooo
287.00,2020-11-4 2:39:54.516

current rise 290.3333333333333
fall 0
offset 0.5361111111110972
newrise 289.7972222222222
nooooooooo
0.00,2020-11-4 2:40:3.465

current rise 289.7972222222222
fall 0
offset 218.8388888888889
newrise 70.95833333333331
nooooooooo
0.00,2020-11-4 2:40:13.413

current rise 70.95833333333331
fall 0
offset 70.95833333333331
newrise 0.0
yeaaaaaaaaaaa
*****


In [274]:
ser.close()

In [275]:
logs_df

,Sig,Name,Turned ON,Init Rise,Curr Rise,Fall,Turned OFF
0,"[[33.0, 268.0, 268.0, 268.0, 268.0, 268.0, 268...",washing machine,2020-11-04 02:39:17.150,290.5,0.0,0,2020-11-04 02:40:13.413000


In [207]:
math.isclose((logs_df['Curr Rise'][0]),(logs_df['Fall'][0]),abs_tol=abs((logs_df['Init Rise'][0])*1/100)) 

True

In [56]:
(pd.to_datetime('2020-11-03 00:28:50.575000'))> pd.to_datetime('2020-11-04 00:28:50.575000')

False

In [61]:
import datetime

In [69]:
pd.to_datetime('2020-11-03 00:28:50.575000') + datetime.timedelta(seconds=3)

Timestamp('2020-11-03 00:28:53.575000')